In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]
# !{sys.executable} -m pip install pip rasterstats 

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-urbanshift' 
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com/data"

In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv(aws_s3_dir + '/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Compute indicator

In [ ]:
cities_indicators_GRE_3_23 = pd.DataFrame() 

In [ ]:
# for i in range(0,len(boundary_georef)):
for i in range(5,6):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']

    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    #read open space
    openspace_path = aws_s3_dir + '/open_space/openstreetmap/v_0/'+boundary_id_aoi+'-OSM-open_space-2022.geojson'
    openspace_geo = requests.get(openspace_path).json()
    openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)
    
    #load population
    pop = ee.ImageCollection('WorldPop/GP/100m/pop')
    pop = (pop.filter(ee.Filter.And(
        ee.Filter.bounds(boundary_geo_ee),
        ee.Filter.inList('year',[2020])))
        .select('population'))
    popImg = pop.mean().rename('population')

    popProj = pop.first().projection()
    popScale = popProj.nominalScale()

    # define threshold distance (m) and buffer open space areas by that to get recreation catchment
    DistanceThres = 400 # meters distance from population to be considered
    def amenityBuffer(feat):
      feat = ee.Feature(feat)
      return feat.buffer(DistanceThres)
    RecCatchment = openspace_geo_ee.map(amenityBuffer)
    RecCatchmentUnion = RecCatchment.union()
    
    #mask population by recreation catchment
    mask = ee.Image.constant(1).clip(RecCatchmentUnion.geometry()).mask()
    popwOSaccess = popImg.updateMask(mask).rename('populationwOpenSpace')
    
    ## add tree cover dataset
    TML = ee.ImageCollection('projects/wri-datalab/TML')
    TreeCoverImg = TML.reduce(ee.Reducer.mean()).rename('b1')
    TreeDataMask = TreeCoverImg.unmask(-99).neq(-99) 
    
    
    # calcs for % population with threshold level (e.g. 10%+) of tree cover within walking distance (e.g. 400m)
    TreePctThreshold = 10 #whole numbers - 0-100, minimum percentage of tree cover threshold to consider 
    circleTheshm = ee.Kernel.circle(DistanceThres, 'meters', False)
    TreeCoverinThreshm = TreeCoverImg.reduceNeighborhood(ee.Reducer.mean(), circleTheshm)
    popwthresTC = popImg.updateMask(TreeCoverinThreshm.gte(TreePctThreshold)).rename('populationwTreeCover')
    
    # combine images 
    combImg = popImg.addBands([popwOSaccess,popwthresTC])
    
    # function to calculate indicators
    def calcs(feat):
        treecoveraccessEq = feat.getNumber('populationwTreeCover').divide(feat.getNumber('population'))
        treecoveraccess = ee.Algorithms.If(feat.getNumber('TreeDataAvailable').eq(0),"NA",treecoveraccessEq)
        openspaceaccess = feat.getNumber('populationwOpenSpace').divide(feat.getNumber('population'))
        return feat.set({
        'PopwOpenSpaceAccessPct': openspaceaccess,
        'PopwTreeCoverAccessPct': treecoveraccess
     }) 

    # use sum reducer to get total populations by features 
    PopbyDistrict=combImg.reduceRegions(reducer= ee.Reducer.sum(), collection=boundary_geo_ee, scale= popScale, tileScale= 4)
    PopbyDistrict = TreeDataMask.reduceRegions(PopbyDistrict,ee.Reducer.anyNonZero().setOutputs(['TreeDataAvailable']),50)

    # # apply function
    PopbyDistrict = PopbyDistrict.map(calcs).select(['geo_id','PopwOpenSpaceAccessPct','PopwTreeCoverAccessPct'])
    
    # store in df and apend
    df = geemap.ee_to_pandas(PopbyDistrict)#,['geo_id','PopwOpenSpaceAccessPct'])
    df = df.rename(columns={"PopwOpenSpaceAccessPct": "GRE_3_2_percentPopwOpenSpaceAccess"}).rename(columns={"PopwTreeCoverAccessPct": "GRE_3_3_percentPopwTreeCoverAccess"})
    cities_indicators_GRE_3_23 = cities_indicators_GRE_3_23.append(df)
    
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # use sum reducer to get total populations by features 
    PopbyDistrict=combImg.reduceRegions(reducer= ee.Reducer.sum(), collection=boundary_geo_ee, scale= popScale, tileScale= 4)
    PopbyDistrict = TreeDataMask.reduceRegions(PopbyDistrict,ee.Reducer.anyNonZero().setOutputs(['TreeDataAvailable']),50)
    
    # apply function
    PopbyDistrict = PopbyDistrict.map(calcs).select(['geo_id','PopwOpenSpaceAccessPct','PopwTreeCoverAccessPct'])
    
    # store in df and apend
    df = geemap.ee_to_pandas(PopbyDistrict)
    df = df.rename(columns={"PopwOpenSpaceAccessPct": "GRE_3_2_percentPopwOpenSpaceAccess"}).rename(columns={"PopwTreeCoverAccessPct": "GRE_3_3_percentPopwTreeCoverAccess"})
    cities_indicators_GRE_3_23 = cities_indicators_GRE_3_23.append(df)
    

In [ ]:
cities_indicators_GRE_3_23

# Workaround for geographies with a lot of open space that won't load into memory - must be run one city at a time

In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv(aws_s3_dir +'/boundaries/v_0/boundary_georef.csv')
boundary_georef

In [ ]:
#delete any existing GEE asset for openspace
ee.data.deleteAsset('users/emackres/thisopenspace')

In [ ]:
## calculate city - adjust range to include single city of interest
for i in range(6,7): #len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']

    #read open space
    openspace_path = aws_s3_dir +'/open_space/openstreetmap/v_0/'+boundary_id_aoi+'-OSM-open_space-2022.geojson'
    openspace_geo = requests.get(openspace_path).json()
    openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)

    exportTask = ee.batch.Export.table.toAsset(
        collection = openspace_geo_ee,
        description = 'description',
        assetId = 'users/emackres/thisopenspace'
    )
    exportTask.start()

In [ ]:
exportTask.status()

In [ ]:
## load saved asset - wait until task status says "COMPLETED"
openspace_geo_ee = ee.FeatureCollection('users/emackres/thisopenspace')

In [ ]:
for i in range(6,7): #len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']

    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # #read open space
    # openspace_path = aws_s3_dir + '/open_space/openstreetmap/v_0/'+boundary_id_aoi+'-OSM-open_space-2022.geojson'
    # openspace_geo = requests.get(openspace_path).json()
    # openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)
    
    #load population
    pop = ee.ImageCollection('WorldPop/GP/100m/pop')
    pop = (pop.filter(ee.Filter.And(
        ee.Filter.bounds(boundary_geo_ee),
        ee.Filter.inList('year',[2020])))
        .select('population'))
    popImg = pop.mean().rename('population')

    popProj = pop.first().projection()
    popScale = popProj.nominalScale()

    # define threshold distance (m) and buffer open space areas by that to get recreation catchment
    DistanceThres = 400 # meters distance from population to be considered
    def amenityBuffer(feat):
      feat = ee.Feature(feat)
      return feat.buffer(DistanceThres)
    RecCatchment = openspace_geo_ee.map(amenityBuffer)
    RecCatchmentUnion = RecCatchment.union()
    
    #mask population by recreation catchment
    mask = ee.Image.constant(1).clip(RecCatchmentUnion.geometry()).mask()
    popwOSaccess = popImg.updateMask(mask).rename('populationwOpenSpace')
    
    ## add tree cover dataset
    TML = ee.ImageCollection('projects/wri-datalab/TML')
    TreeCoverImg = TML.reduce(ee.Reducer.mean()).rename('b1')
    TreeDataMask = TreeCoverImg.unmask(-99).neq(-99) 
    
    
    # calcs for % population with threshold level (e.g. 10%+) of tree cover within walking distance (e.g. 400m)
    TreePctThreshold = 10 #whole numbers - 0-100, minimum percentage of tree cover threshold to consider 
    circleTheshm = ee.Kernel.circle(DistanceThres, 'meters', False)
    TreeCoverinThreshm = TreeCoverImg.reduceNeighborhood(ee.Reducer.mean(), circleTheshm)
    popwthresTC = popImg.updateMask(TreeCoverinThreshm.gte(TreePctThreshold)).rename('populationwTreeCover')
    
    # combine images 
    combImg = popImg.addBands([popwOSaccess,popwthresTC])
    
    # function to calculate indicators
    def calcs(feat):
        treecoveraccessEq = feat.getNumber('populationwTreeCover').divide(feat.getNumber('population'))
        treecoveraccess = ee.Algorithms.If(feat.getNumber('TreeDataAvailable').eq(0),"NA",treecoveraccessEq)
        openspaceaccess = feat.getNumber('populationwOpenSpace').divide(feat.getNumber('population'))
        return feat.set({
        'PopwOpenSpaceAccessPct': openspaceaccess,
        'PopwTreeCoverAccessPct': treecoveraccess
     }) 

    # use sum reducer to get total populations by features 
    PopbyDistrict=combImg.reduceRegions(reducer= ee.Reducer.sum(), collection=boundary_geo_ee, scale= popScale, tileScale= 4)
    PopbyDistrict = TreeDataMask.reduceRegions(PopbyDistrict,ee.Reducer.anyNonZero().setOutputs(['TreeDataAvailable']),50)

    # # apply function
    PopbyDistrict = PopbyDistrict.map(calcs).select(['geo_id','PopwOpenSpaceAccessPct','PopwTreeCoverAccessPct'])
    
    # store in df and apend
    df = geemap.ee_to_pandas(PopbyDistrict)#,['geo_id','PopwOpenSpaceAccessPct'])
    df = df.rename(columns={"PopwOpenSpaceAccessPct": "GRE_3_2_percentPopwOpenSpaceAccess"}).rename(columns={"PopwTreeCoverAccessPct": "GRE_3_3_percentPopwTreeCoverAccess"})
    cities_indicators_GRE_3_23 = cities_indicators_GRE_3_23.append(df)
    
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # use sum reducer to get total populations by features 
    PopbyDistrict=combImg.reduceRegions(reducer= ee.Reducer.sum(), collection=boundary_geo_ee, scale= popScale, tileScale= 4)
    PopbyDistrict = TreeDataMask.reduceRegions(PopbyDistrict,ee.Reducer.anyNonZero().setOutputs(['TreeDataAvailable']),50)
    
    # apply function
    PopbyDistrict = PopbyDistrict.map(calcs).select(['geo_id','PopwOpenSpaceAccessPct','PopwTreeCoverAccessPct'])
    
    # store in df and apend
    df = geemap.ee_to_pandas(PopbyDistrict)
    df = df.rename(columns={"PopwOpenSpaceAccessPct": "GRE_3_2_percentPopwOpenSpaceAccess"}).rename(columns={"PopwTreeCoverAccessPct": "GRE_3_3_percentPopwTreeCoverAccess"})
    cities_indicators_GRE_3_23 = cities_indicators_GRE_3_23.append(df)
    

In [ ]:
cities_indicators_GRE_3_23

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv('https://'+bucket_name+'.s3.eu-west-3.amazonaws.com/indicators/cities_indicators_ericV1.csv')
cities_indicators

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    else:
        print("add new indicators")
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    return(cities_indicators_df)

In [ ]:
cities_indicators_GRE_3_2 = cities_indicators_GRE_3_23[["geo_id","GRE_3_2_percentPopwOpenSpaceAccess"]]
cities_indicators_GRE_3_3 = cities_indicators_GRE_3_23[["geo_id","GRE_3_3_percentPopwTreeCoverAccess"]]

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = cities_indicators_GRE_3_2,
                                            indicator_name = "GRE_3_2_percentPopwOpenSpaceAccess")

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators_merged,
                                            new_indicator_table = cities_indicators_GRE_3_3,
                                            indicator_name = "GRE_3_3_percentPopwTreeCoverAccess")

In [ ]:
cities_indicators_merged

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
# aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'indicators/cities_indicators_ericV1.csv'

cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')